<a href="https://colab.research.google.com/github/Vinther901/Corona/blob/master/Model_Tester_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade torch-scatter
!pip install --upgrade torch-sparse
!pip install --upgrade torch-cluster
!pip install --upgrade torch-spline-conv 
!pip install torch-geometric

  Created wheel for torch-scatter: filename=torch_scatter-2.0.5-cp36-cp36m-linux_x86_64.whl size=12183747 sha256=b01422e1f6c7f9b21062755dac7409bd10c2d32a1e5bc548a5959d9b94b4efd8
  Stored in directory: /root/.cache/pip/wheels/42/59/d3/9866e4fd8e1fe9260740acfe22322c428bc0dc064d3ebc456c
Successfully built torch-scatter
  Created wheel for torch-sparse: filename=torch_sparse-0.6.6-cp36-cp36m-linux_x86_64.whl size=20883463 sha256=9955a0c37e08d2f7340606ffa21fdd8a6ba721688ab7aef5d5c43b5ddc841335
  Stored in directory: /root/.cache/pip/wheels/2a/09/04/4d83555faac0525a67b4f00c679afb65b44a03976831b49750
Successfully built torch-sparse
  Created wheel for torch-cluster: filename=torch_cluster-1.5.5-cp36-cp36m-linux_x86_64.whl size=21912820 sha256=6446f35533e431d4f256af0e22a19b29cb85c1c883580b3afd9bb5c8e6f623bd
  Stored in directory: /root/.cache/pip/wheels/2c/7d/19/b795f11b2214aba22ec3ecc13725b654d7d1206af12feef2a6
Successfully built torch-cluster
  Created wheel for torch-spline-conv: filename=t

In [113]:
from torch_geometric.data import DataLoader, InMemoryDataset
import torch
import time
import os
import numpy as np

In [114]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [115]:
path = '/content/drive/My Drive/Colab Notebooks'
os.listdir(path+'/copy_dataset')

['processed']

Load dataset, Nodes as doms with edges to 6 nearest neighbours

In [116]:
#The @properties should be unimportant for now, including process since the data is processed.
class MakeDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(MakeDataset, self).__init__(root, transform, pre_transform)
        # print(self.processed_paths)
        self.data, self.slices = torch.load(self.processed_paths[0]) #Loads PROCESSED.file
                                                                      #perhaps check print(self.processed_paths)
    @property
    def raw_file_names(self):
        return os.listdir(path)

    @property
    def processed_file_names(self):
        return os.listdir(path+'/copy_dataset/processed')

    def process(self):
        pass

dataset = MakeDataset(root = path + '/copy_dataset')

#### Changing target variables to one hot encoded (or not) neutrino type ####
#### It is important to remember to change the slicing as well as y#
types = torch.tensor([np.zeros(100000),np.ones(100000),np.ones(100000)*2],dtype=torch.int64).reshape((1,-1))

dataset.data.y = types[0]
dataset.slices['y'] = torch.tensor(np.arange(300000+1))

Dividing up the dataset

In [117]:
dataset = dataset.shuffle()

# train_dataset = dataset[:50000]
# val_dataset = dataset[50000:75000]
# test_dataset = dataset[75000:100000]

train_dataset = dataset[:200000]
val_dataset = dataset[200000:250000]
test_dataset = dataset[250000:]

In [118]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Define model.

In [124]:
from torch_geometric.nn import  DNAConv #GINConv, SGConv, SAGPooling, GCNConv, GATConv
from torch_scatter import scatter_add
import torch.nn.functional as F

# Taken from: https://github.com/rusty1s/pytorch_geometric/blob/master/examples/dna.py

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        in_channels = 5
        hidden_channels = 30
        self.hidden_channels = hidden_channels
        num_layers = 10

        self.lin1 = torch.nn.Linear(in_channels,hidden_channels)
        self.DNAConvs = torch.nn.ModuleList()
        for i in range(num_layers):
            self.DNAConvs.append(DNAConv(hidden_channels,heads=3,groups=1,dropout=0))
        self.lin2 = torch.nn.Linear(hidden_channels,3)

    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = F.relu(self.lin1(x))
        # x = F.dropout(x,p=0.5,training.self.training)
        x_all = x.view(-1,1,self.hidden_channels)
        for conv in self.DNAConvs:
            x = F.relu(conv(x_all,edge_index))
            x = x.view(-1,1,self.hidden_channels)
            x_all = torch.cat([x_all,x],dim=1)
        x = x_all[:,-1]
        x = scatter_add(x,data.batch,dim=0)
        # x = F.dropout(x,p=0.5,training.self.training)
        x = self.lin2(x)
        return torch.log_softmax(x,dim=1)

In [125]:
device = torch.device('cuda')
model = Net().to(device)
# optimizer = torch.optim.SGD(model.parameters(),lr=0.00001,momentum=0.8,nesterov=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

crit = torch.nn.NLLLoss()   #Loss function

In [132]:
def train():
    model.train()
    for data in train_loader:
        label = data.y.to(device)
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = crit(output, label)
        print(loss.item(), (output.argmax(dim=1).eq(label).sum().float()/len(label)).item())
        loss.backward()
        optimizer.step()

    return #loss, correct.float()/len(train_dataset)

In [133]:
loss_list, ratio_list = [], []
def epochs(i):
    print('Begins training')
    t = time.time()
    for epoch in range(i):
        print(f'Epoch: {epoch}')
        train()
        # curr_loss,ratio = train()
        # loss_list.append(curr_loss.item())
        # ratio_list.append(ratio.item())
        # print(curr_loss.item(),ratio.item())
        print(f'time since beginning: {time.time() - t}')
    print('Done')

In [134]:
epochs(1)

Streaming output truncated to the last 5000 lines.
14.786277770996094 0.1875
28.23196029663086 0.3125
43.97823715209961 0.5625
46.32178497314453 0.375
103.36266326904297 0.25
39.85849380493164 0.4375
47.57851028442383 0.3125
55.36578369140625 0.375
49.79048538208008 0.3125
91.23604583740234 0.1875
72.5684814453125 0.375
71.7454605102539 0.25
59.68378448486328 0.3125
26.57715606689453 0.3125
15.640586853027344 0.4375
51.360450744628906 0.1875
20.628002166748047 0.375
58.389957427978516 0.125
27.350357055664062 0.3125
56.21746063232422 0.25
72.79116821289062 0.25
38.94485092163086 0.4375
11.90311050415039 0.3125
60.33665084838867 0.4375
60.8297119140625 0.1875
101.7776870727539 0.1875
42.12721633911133 0.5
30.335891723632812 0.375
85.51593017578125 0.375
109.92091369628906 0.3125
23.580177307128906 0.3125
48.277435302734375 0.3125
161.78077697753906 0.3125
75.58753967285156 0.4375
85.52899169921875 0.125
30.956687927246094 0.5
7.110797882080078 0.4375
27.141143798828125 0.4375
25.1584167

In [135]:
epochs(1)

Streaming output truncated to the last 5000 lines.
1.0941762924194336 0.25
1.0927232503890991 0.5
1.0962573289871216 0.25
1.098311424255371 0.3125
1.107133150100708 0.25
1.0953019857406616 0.375
1.0993291139602661 0.3125
1.1007976531982422 0.3125
1.1034232378005981 0.125
1.0976901054382324 0.4375
1.094333291053772 0.3125
1.092756986618042 0.375
1.0971932411193848 0.5
1.0958914756774902 0.4375
1.0956521034240723 0.4375
1.100834608078003 0.1875
1.095894694328308 0.1875
1.0867291688919067 0.5
1.098714828491211 0.3125
1.1016008853912354 0.3125
1.1035094261169434 0.3125
1.0966137647628784 0.3125
1.0902462005615234 0.4375
1.1005489826202393 0.375
1.0929421186447144 0.375
1.109900951385498 0.1875
1.0971953868865967 0.5
1.0971872806549072 0.375
1.099977970123291 0.375
1.0970951318740845 0.375
1.0966087579727173 0.3125
1.0900977849960327 0.5
1.0966289043426514 0.375
1.0976991653442383 0.4375
1.0957869291305542 0.5
1.0965873003005981 0.25
1.096933126449585 0.3125
1.1008853912353516 0.375
1.09931

Try and estimate time spent to find good maxima

In [112]:
max_batch_size = 16
train_sizes = [2**n for n in range(2,15)]

acc_benchmark = 0.8

times = np.zeros(len(train_sizes),astype=float)
num_epochs = np.zeros(len(train_sizes),astype=int)

for i in range(len(train_sizes)):

  model = Net().to(device)
  # optimizer = torch.optim.SGD(model.parameters(),lr=0.1,momentum=0.9, dampening=0.0,nesterov = True)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

  train_size = train_sizes[i]
  batch_size = min(max_batch_size,train_size)
  train_dataset = dataset[:train_size]
  train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)

  t0 = time.time()

  num_epoch = 0
  acc = 0.0
  while acc < acc_benchmark:
    num_epoch += 1
    acc = 0.0
    for data in train_loader:
        data = data.to(device)
        label = data.y
        optimizer.zero_grad()
        output = model(data)
        loss = crit(output, label)
        loss.backward()
        optimizer.step()
        acc += output.argmax(dim=1).eq(label).sum()
    acc /= train_size
    if num_epoch % 10 == 0:
      print(f'Train size: {train_size}, loss: {loss.item()}, current accuracy: {acc}, epochs: {num_epoch}')

  print(f'Final: Train size: {train_size}, current accuracy: {acc}, epochs: {num_epoch}')
  times[i] = time.time() - t0
  num_epochs[i] = num_epoch

Train size: 4, loss: 21281.75390625, current accuracy: 0.25, epochs: 10
Train size: 4, loss: 4040.525390625, current accuracy: 0.5, epochs: 20
Train size: 4, loss: 5636.3251953125, current accuracy: 0.5, epochs: 30
Train size: 4, loss: 2412.24462890625, current accuracy: 0.5, epochs: 40
Train size: 4, loss: 255.676513671875, current accuracy: 0.5, epochs: 50
Train size: 4, loss: 367.357177734375, current accuracy: 0.75, epochs: 60
Train size: 4, loss: 239.330810546875, current accuracy: 0.25, epochs: 70
Train size: 4, loss: 134.376708984375, current accuracy: 0.5, epochs: 80
Train size: 4, loss: 2.635009765625, current accuracy: 0.75, epochs: 90
Train size: 4, loss: 240.994873046875, current accuracy: 0.75, epochs: 100
Train size: 4, loss: 132.90234375, current accuracy: 0.5, epochs: 110
Train size: 4, loss: 149.969482421875, current accuracy: 0.5, epochs: 120
Train size: 4, loss: 44.10595703125, current accuracy: 0.75, epochs: 130
Train size: 4, loss: 50.64697265625, current accuracy:

KeyboardInterrupt: ignored

In [110]:
times, num_epochs

(array([ 102.7023735, 4932.       ,    0.       ,    0.       ,
           0.       ,    0.       ,    0.       ,    0.       ,
           0.       ,    0.       ,    0.       ,    0.       ,
           0.       ]),
 array([4911.        ,    0.        ,  105.84280539,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ]))